In [33]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot


In [35]:
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))

    opt = Adam(lr=0.0002,beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model



print("Descrimintor")

Descrimintor


In [36]:
def define_generator(latent_dim):
    model = Sequential()
    n_node = 128 *7 *7
    model.add(Dense(n_node, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7,7,128)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(1, (7,7), activation="sigmoid", padding="same"))
    return model
    
    
print("Generator")

Generator


In [37]:
def define_gan(g_model, d_model):
    d_model.trainable = False

    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

print("Model")

Model


In [38]:
# load and prepare mnist training images
def load_real_samples():
    (trainX,_),(_,_) = load_data()
    X = expand_dims(trainX, axis = -1)
    X=X.astype('float32')
    X=X/255.0
    return X

# select real samples
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples,1))
    return X,y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model,latent_dim,n_samples):
    x_input = generate_latent_points(latent_dim,n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples,1))
    return X,y

# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        pyplot.subplot(n,n,1+i)
        pyplot.axis('off')
        pyplot.imshow(examples[i,:,:,0], cmap='gray_r')
    
    filename = 'generated_plot_ep_%03d.png'%(epoch+1)
    pyplot.savegif(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real,y_real, verbose=0)
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose=0)

    print("Accuracy real: %.0f%%, fake: %.0f%%" % (acc_real*100, acc_fake*100))
    save_plot(X_fake, epoch)

    filename = 'generator_model_%03d.h5' %(epoch+1)
    g_model.save(filename)
    

    
print("Before training")

Before training


In [39]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epoch = int(dataset.shape[0]/n_batch)
    half_batch = int(n_batch/2)

    for i in range(n_epochs):
        for j in range(bat_per_epoch):

            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X,y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss,_ = d_model.train_on_batch(X,y)

            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch,1))

            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            print(">epoch%d, %d/%d, d_loss=%.3f, g_loss=%.3f" %(i+1,j+1,bat_per_epoch,d_loss,g_loss))
        
        if(i+1) % 10 ==10:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

            
            
            
print("After training")

After training


In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim)

4/4 [==============================] - 1s 84ms/step



>epoch1, 1/234, d_loss=0.708, g_loss=0.819
4/4 [==============================] - 0s 81ms/step
>epoch1, 2/234, d_loss=0.657, g_loss=0.930
4/4 [==============================] - 0s 85ms/step
>epoch1, 3/234, d_loss=0.640, g_loss=0.969
4/4 [==============================] - 0s 82ms/step
>epoch1, 4/234, d_loss=0.646, g_loss=0.877
4/4 [==============================] - 0s 80ms/step
>epoch1, 5/234, d_loss=0.662, g_loss=0.756
4/4 [==============================] - 0s 79ms/step
>epoch1, 6/234, d_loss=0.659, g_loss=0.697
4/4 [==============================] - 0s 92ms/step
>epoch1, 7/234, d_loss=0.623, g_loss=0.685
4/4 [==============================] - 1s 106ms/step
>epoch1, 8/234, d_loss=0.579, g_loss=0.689
4/4 [==============================] - 0s 119ms/step
>epoch1, 9/234, d_loss=0.536, g_loss=0.695
4/4 [==============================] - 0s 86ms/step
>epoch1, 10/234, d_loss=0.487, g_loss=0.701
4/4 [==============================] - 0s 82ms/step
>epoch1, 11/234, d_loss=0.444, g_loss=0.709
4/4

4/4 [==============================] - 0s 96ms/step
>epoch1, 87/234, d_loss=0.589, g_loss=0.903
4/4 [==============================] - 0s 99ms/step
>epoch1, 88/234, d_loss=0.593, g_loss=1.034
4/4 [==============================] - 0s 91ms/step
>epoch1, 89/234, d_loss=0.571, g_loss=0.999
4/4 [==============================] - 0s 94ms/step
>epoch1, 90/234, d_loss=0.571, g_loss=0.972
4/4 [==============================] - 0s 98ms/step
>epoch1, 91/234, d_loss=0.574, g_loss=1.059
4/4 [==============================] - 0s 100ms/step
>epoch1, 92/234, d_loss=0.562, g_loss=0.985
4/4 [==============================] - 0s 91ms/step
>epoch1, 93/234, d_loss=0.575, g_loss=0.986
4/4 [==============================] - 0s 104ms/step
>epoch1, 94/234, d_loss=0.534, g_loss=0.996
4/4 [==============================] - 0s 93ms/step
>epoch1, 95/234, d_loss=0.544, g_loss=0.987
4/4 [==============================] - 0s 91ms/step
>epoch1, 96/234, d_loss=0.526, g_loss=1.035
4/4 [==============================] -